# Setup

In [ ]:
import CvCmdApi
CvCmder = CvCmdApi.CvCmdHandler('COM19', 1280, 720, 100)

# Demo

In [ ]:
import keyboard
import time
from ipywidgets import widgets
gimbal_coordinate_x_Slider = widgets.IntSlider(min=0,max=640,step=1,description='coordinate_x:',value=0)
gimbal_coordinate_y_Slider = widgets.IntSlider(min=0,max=480,step=1,description='coordinate_y:',value=0)
chassis_speed_x_Slider = widgets.FloatSlider(min=-1,max=1,step=0.01,description='speed_x:',value=0)
chassis_speed_y_Slider = widgets.FloatSlider(min=-1,max=1,step=0.01,description='speed_y:',value=0)
display(gimbal_coordinate_x_Slider)
display(gimbal_coordinate_y_Slider)
display(chassis_speed_x_Slider)
display(chassis_speed_y_Slider)

In [ ]:
oldflags = (False, False, False)
counter = 0
loop_delay = 0.05
while True:
    flags = CvCmder.CvCmd_Heartbeat(gimbal_coordinate_x_Slider.value + 1280, gimbal_coordinate_y_Slider.value + 720, chassis_speed_x_Slider.value, chassis_speed_y_Slider.value)
    if flags != oldflags:
        oldflags = flags
        print(flags)
    if counter == int(5/loop_delay):
        CvCmder.CvCmd_Shoot()
    counter += 1
    time.sleep(loop_delay)

In [ ]:
print(CvCmder.ShootSwitch)

In [ ]:
oldflags = (False, False, False)
while True:
    if keyboard.is_pressed('s'):
        chassis_speed_y_Slider.value += -0.02
    elif keyboard.is_pressed('w'):
        chassis_speed_y_Slider.value += 0.02
    elif keyboard.is_pressed('a'):
        chassis_speed_x_Slider.value += -0.02
    elif keyboard.is_pressed('d'):
        chassis_speed_x_Slider.value += 0.02
    elif keyboard.is_pressed('q'):
        break

    flags = CvCmder.CvCmd_Heartbeat(gimbal_coordinate_x=chassis_speed_x_Slider.value+1280,gimbal_coordinate_y=chassis_speed_y_Slider.value+720,chassis_speed_x=chassis_speed_x_Slider.value,chassis_speed_y=chassis_speed_y_Slider.value)
    if flags != oldflags:
        oldflags = flags
        print(flags)
    time.sleep(0.050)

In [ ]:
CvCmder.ser.close()

In [ ]:
CvCmder.ser.open()

In [ ]:
CvCmder.ser.read(CvCmder.ser.in_waiting)

In [ ]:
CvCmder.CvCmd_Reset()

## Unit test

In [ ]:
counter = 0
while counter <= 6:
    CvCmder.ser.write(CvCmder.txAckMsg)
    counter += 1
    time.sleep(0.1)

In [ ]:
(gimbal_coordinate_x, gimbal_coordinate_y, chassis_speed_x, chassis_speed_y) = (300, 400, 0, 0)
CvCmder.CvCmd_ConditionSignals()
CvCmder.txCvCmdMsg[CvCmder.DATA_PAYLOAD_INDEX:CvCmder.DATA_PAYLOAD_INDEX+16] = struct.pack('<ffff', CvCmder.gimbal_cmd_delta_yaw, CvCmder.gimbal_cmd_delta_pitch, CvCmder.chassis_cmd_speed_x, CvCmder.chassis_cmd_speed_y)
# test min separation time
CvCmder.ser.write(CvCmder.txAckMsg)
while True:
    CvCmder.ser.write(CvCmder.txCvCmdMsg)
    time.sleep(0.002)

In [ ]:
# incomplete/invalid msg
CvCmder.ser.write(CvCmder.txAckMsg[:10])
CvCmder.ser.write(CvCmder.txAckMsg[5:])

In [ ]:
# Find corrupted package
stop = False
while ~stop:
    bytesRead = CvCmder.ser.read(CvCmder.ser.in_waiting)
    dataPackets = re.findall(CvCmder.eSepChar.CHAR_STX.value + CvCmder.eMsgType.MSG_MODE_CONTROL.value + b"." + CvCmder.eSepChar.CHAR_UNUSED.value + b"{11}" + CvCmder.eSepChar.CHAR_ETX.value, bytesRead)
    for i in dataPackets:
        if i !=b'\x3E\x3E\x10\x02\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff':
            print(dataPackets)
            stop = True
            break